# Developping RK4 integrator

In [1]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
import joptimise
using Printf

In [2]:
include("../../julia-r3bp/R3BP/src/R3BP.jl")
include("../src/SailorMoon.jl")   # relative path to main file of module

Main.SailorMoon

In [20]:
plotly()

Plots.PlotlyBackend()

In [3]:
param3b = SailorMoon.dyanmics_parameters()
lps = SailorMoon.lagrange_points(param3b.mu2)

5×6 Matrix{Float64}:
  0.836915   0.0       0.0  0.0  0.0  0.0
  1.15568    0.0       0.0  0.0  0.0  0.0
 -1.00506    0.0       0.0  0.0  0.0  0.0
  0.487849   0.866025  0.0  0.0  0.0  0.0
  0.487849  -0.866025  0.0  0.0  0.0  0.0

In [4]:
lp = 2
Az_km = 1200.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")
res.flag

Halo guess Az_km: 1200.0


1

In [26]:
# solver settings within fitness function
# https://diffeq.sciml.ai/stable/solvers/dynamical_solve/#Symplectic-Integrators
method = Tsit5()  # CalvoSanz4()
reltol = 1e-10
abstol = 1e-10
#dt = 0.005

param3b = SailorMoon.dyanmics_parameters()
lps = SailorMoon.lagrange_points(param3b.mu2)

lp = 2
Az_km = 1200.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")

x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, method, reltol=reltol, abstol=abstol)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix
ys0 = R3BP.get_eigenvector(monodromy, true, 1);

# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, Tsit5(), 1e-12, 1e-12
);

Halo guess Az_km: 1200.0
Linear stability ν = 618.7618470241374


In [27]:
LPOArrival.x0

6-element Vector{Float64}:
  1.1226879860196828
  0.0
 -5.228885624166955e-26
  0.0
  0.1650993133797659
  0.0

In [96]:
tmax_si = 0.3  # N
isp_si = 3500  # sec
mdot_si = tmax_si / (isp_si * 9.81)
mstar = 2000  # kg
rp_parking = (6378+200)/param3b.lstar   # parking orbit radius

tmax = AstrodynamicsBase.dimensional2canonical_thrust(
    tmax_si, mstar, param3b.lstar, param3b.tstar
)
mdot = AstrodynamicsBase.dimensional2canonical_mdot(
    mdot_si, mstar, param3b.tstar
)

params = [
    param3b.mu2, param3b.mus, deg2rad(192), param3b.as, param3b.oms, 
    0.0, 0.0, 0.0, 0.0, 0.0,
    SailorMoon.dv_sun_dir_angles
]

# construct final state
svf = vcat(
    SailorMoon.set_terminal_state(0.4, param3b, LPOArrival), 
    1.0
)

_prob_base = ODEProblem(
    SailorMoon.rhs_bcr4bp_emframe_thrust!, svf, [0,-25], params
);

In [97]:
sol = DifferentialEquations.solve(_prob_base, method, reltol=reltol, abstol=abstol);

In [402]:
# store the apoapsis value
function apoapsis_cond(u,t,int)
    # condition 1: SC is sufficiently far from the moon
    r = sqrt((u[1] - (1-param3b.mu2))^2 + u[2]^2 + u[3]^2) # SC-Moon distance
    moon_soi = 5000 / param3b.lstar # define "sphere of influence"
    
    if sqrt(u[1]^2 + u[2]^2 + u[3]^2) > 2.0
        # condition 2: dot product of velocity and position is zero
        return dot((u[1:3] + [param3b.mu2, 0.0, 0.0]), u[4:6])
    else
        return NaN
    end
end

# store the periapsis value and terminate
function periapsis_cond(u,t,int)
    r = sqrt((u[1] + param3b.mu2)^2 + u[2]^2 + u[3]^2)  # SC-earth distance
    earth_leo_ub = 10000 / param3b.lstar  # km
    earth_leo_lb = 200  / param3b.lstar  # km
    
    if earth_leo_lb < r < earth_leo_ub
        return dot((u[1:3] + [param3b.mu2, 0.0, 0.0]), u[4:6])
    else 
        return NaN
    end
end

terminate_affect!() = true
no_affect!() = false

# include callback functions 
apoapsis_cb  = ContinuousCallback(apoapsis_cond, terminate_affect!)
periapsis_cb = ContinuousCallback(periapsis_cond, terminate_affect!)
cbs = [apoapsis_cb, periapsis_cb];

In [403]:
apoapsis_cb.affect!()

true

In [416]:
include("../src/SailorMoon.jl")

Main.SailorMoon

In [417]:
cbs[1].affect!()

true

In [418]:
val = NaN
isnan(val)

true

In [419]:
res = SailorMoon.integrate_rk4(_prob_base, 0.01, cbs);
res.retcode

:PrematureEnd

In [420]:
res.event_states[1]

1-element Vector{Any}:
 [-3.2120624257635404, 2.531778912512784, 1.0065055870387817e-21, 2.456690893329257, 3.105505321831814, 5.891135646123419e-22, 1.0]

In [421]:
hcat(res.u...)

7×1358 Matrix{Float64}:
  1.17198      1.17154       1.1711       …  -3.18734      -3.21206
  0.0471678    0.0483678     0.049553         2.56271       2.53178
  1.3735e-21   1.36099e-21   1.34815e-21      1.0124e-21    1.00651e-21
  0.0429353    0.0437823     0.0446072        2.48677       2.45669
 -0.120747    -0.119266     -0.117751         3.08094       3.10551
  1.2352e-21   1.26761e-21   1.29977e-21  …   5.88908e-22   5.89114e-22
  1.0          1.0           1.0              1.0           1.0

In [422]:
function circle_coordinates(radius,center,steps::Int=100)
    thetas = LinRange(0,2π,steps)
    coord = zeros(2,steps)
    for (i,theta) in enumerate(thetas)
        coord[1,i] = radius*cos(theta) + center[1]
        coord[2,i] = radius*sin(theta) + center[2]
    end
    return coord
end

circle_coordinates (generic function with 2 methods)

In [423]:
earth_coord = circle_coordinates(6378/param3b.lstar, [-param3b.mu2,0]);
moon_coord  = circle_coordinates(1737/param3b.lstar, [1-param3b.mu2,0]);

In [424]:
r2s = []
for i = 1:length(Array(sol)[1,:])
    push!(r2s, norm(Array(sol)[1:3, i] - [-param3b.mu2, 0, 0]))
end
minimum(r2s) * param3b.lstar - 6378

444.24385905157305

In [425]:
# plot
ptraj = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.2)

# comparing propagations
plot!(ptraj, Array(sol)[1,:], Array(sol)[2,:], color=:dodgerblue, label="Tsit5")
plot!(ptraj, hcat(res.u...)[1,:], hcat(res.u...)[2,:], color=:deeppink, label="RK4")

scatter!(ptraj, lps[:,1], lps[:,2], marker=:diamond, color=:red, label="LPs")
plot!(earth_coord[1,:], earth_coord[2,:], color=:blue, label="Earth")
plot!(moon_coord[1,:], moon_coord[2,:], color=:black, label="Moon")
ptraj